In [7]:
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, set_seed
import torch
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR
from torch.utils.data import DataLoader
from sklearn.model_selection import KFold
import nlpaug.augmenter.word as naw

# 定义超参数
BATCH_SIZE = 32  # 批处理大小
EPOCHS = 70  # 迭代次数
MAX_LENGTH = 40  # 序列最大长度
LEARNING_RATE = 5e-5  # 学习率
WEIGHT_DECAY = 0.01  # 权重衰减系数
MODEL_NAME = 'bert-base-chinese'  # 预训练模型名
N_FOLDS = 7  # 交叉验证折数
RANDOM_STATE = 42  # 随机种子

# 设置随机种子，保证结果可复现
set_seed(RANDOM_STATE)

# 加载csv数据集，数据路径为'train_dataset.csv'
raw_dataset = load_dataset('csv', data_files='train_dataset.csv', split='train')

# 对标签进行编码，将类别标签转化为数字
raw_dataset = raw_dataset.class_encode_column('label')

# 从预训练模型名加载Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# 定义编码函数，对输入的文本进行编码
def encode(examples):
    return tokenizer(examples['query'], truncation=True, padding='max_length', max_length=MAX_LENGTH)

# 对数据集进行编码
raw_dataset = raw_dataset.map(encode, batched=True, remove_columns='#')

# 数据增强
augmenter = naw.ContextualWordEmbsAug(model_path='bert-base-chinese', action='substitute')

# 编写数据增强函数
def augment_text(example):
    example['query'] = augmenter.augment(example['query'])
    return example

# 对数据集进行增强
augmented_dataset = raw_dataset.map(augment_text)

# 创建交叉验证对象
cross_validator = KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE)

Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-2de56802f1ede12a/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-2de56802f1ede12a/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-87761e9464147f5e.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-2de56802f1ede12a/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-08941be77ca52344.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-2de56802f1ede12a/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-a3e7b026a12aab60.arrow


In [8]:
# 进行交叉验证
for fold, (train_indices, valid_indices) in enumerate(cross_validator.split(raw_dataset)):
    # 划分训练集和验证集
    train_subset = raw_dataset.select(train_indices)
    valid_subset = raw_dataset.select(valid_indices)

    # 设置数据格式，用于训练模型
    train_subset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
    valid_subset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

    # 创建数据加载器
    train_dataloader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True)
    valid_dataloader = DataLoader(valid_subset, batch_size=BATCH_SIZE, shuffle=False)

    # 准备模型，优化器和学习率调度器
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=train_subset.features['label'].num_classes)
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model.to(device)

    optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = OneCycleLR(optimizer, max_lr=LEARNING_RATE, epochs=EPOCHS, steps_per_epoch=len(train_dataloader))

    # 进行模型训练
    best_acc = 0
    for epoch in range(EPOCHS):
        total_loss = 0
        model.train()
        for step, batch in enumerate(train_dataloader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            # 模型前向传播，计算loss
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            # 反向传播，更新梯度
            loss.backward()

            # 优化器步进，更新模型参数
            optimizer.step()
            # 更新学习率
            scheduler.step()
            # 清空梯度
            optimizer.zero_grad()

            total_loss += loss.detach().item()

        # 验证模型，计算验证集上的精度
        model.eval()
        valid_predictions = []
        for batch in valid_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            with torch.no_grad():
                outputs = model(input_ids, attention_mask=attention_mask)

            valid_predictions.extend(outputs.logits.argmax(-1).cpu().numpy())

        valid_acc = np.mean(valid_predictions == valid_subset['label'].numpy())
        print(f"Fold: {fold+1}, Epoch: {epoch+1}, Train Loss: {total_loss:.3f}, Validation Accuracy: {valid_acc:.4f}")

        # 保存在验证集上最好的结果
        if valid_acc >= best_acc:
            model.save_pretrained('output')

    # 加载测试数据集，对测试集进行预测
    test_dataset = load_dataset('csv', data_files='test_dataset.csv', split='train')
    test_id = test_dataset['id']
    test_dataset = test_dataset.map(encode, batched=True, remove_columns='id')
    test_dataset.set_format('torch', columns=['input_ids', 'attention_mask'])
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    model = AutoModelForSequenceClassification.from_pretrained('output')
    model.to(device)

    model.eval()
    predictions = []
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)

        predictions.extend(outputs.logits.argmax(-1).cpu().numpy())

    # 将预测的标签从数字转换回原始的类别
    predicted_labels = train_subset.features['label'].int2str(predictions)

    # 保存预测结果，以csv格式存储
    submission = pd.DataFrame({'id': test_id, 'label': predicted_labels})
    submission.to_csv(f'nlpaug_fold_{fold+1}.csv', index=False)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Fold: 1, Epoch: 1, Train Loss: 465.677, Validation Accuracy: 0.0026
Fold: 1, Epoch: 2, Train Loss: 461.894, Validation Accuracy: 0.0026
Fold: 1, Epoch: 3, Train Loss: 457.486, Validation Accuracy: 0.0000
Fold: 1, Epoch: 4, Train Loss: 452.667, Validation Accuracy: 0.0053
Fold: 1, Epoch: 5, Train Loss: 442.106, Validation Accuracy: 0.0211
Fold: 1, Epoch: 6, Train Loss: 429.361, Validation Accuracy: 0.0342
Fold: 1, Epoch: 7, Train Loss: 413.583, Validation Accuracy: 0.0658
Fold: 1, Epoch: 8, Train Loss: 398.047, Validation Accuracy: 0.1132
Fold: 1, Epoch: 9, Train Loss: 378.273, Validation Accuracy: 0.1632
Fold: 1, Epoch: 10, Train Loss: 359.021, Validation Accuracy: 0.2158
Fold: 1, Epoch: 11, Train Loss: 336.840, Validation Accuracy: 0.2816
Fold: 1, Epoch: 12, Train Loss: 311.819, Validation Accuracy: 0.3368
Fold: 1, Epoch: 13, Train Loss: 287.410, Validation Accuracy: 0.4026
Fold: 1, Epoch: 14, Train Loss: 261.648, Validation Accuracy: 0.4342
Fold: 1, Epoch: 15, Train Loss: 234.988, Va

Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-67caf77471a5a48e/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-67caf77471a5a48e/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-dcc4d91e0724ffa4.arrow
Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertFor

Fold: 2, Epoch: 1, Train Loss: 466.056, Validation Accuracy: 0.0026
Fold: 2, Epoch: 2, Train Loss: 461.704, Validation Accuracy: 0.0026
Fold: 2, Epoch: 3, Train Loss: 457.623, Validation Accuracy: 0.0026
Fold: 2, Epoch: 4, Train Loss: 450.508, Validation Accuracy: 0.0079
Fold: 2, Epoch: 5, Train Loss: 440.777, Validation Accuracy: 0.0132
Fold: 2, Epoch: 6, Train Loss: 428.598, Validation Accuracy: 0.0263
Fold: 2, Epoch: 7, Train Loss: 414.364, Validation Accuracy: 0.0579
Fold: 2, Epoch: 8, Train Loss: 396.794, Validation Accuracy: 0.0842
Fold: 2, Epoch: 9, Train Loss: 377.130, Validation Accuracy: 0.1474
Fold: 2, Epoch: 10, Train Loss: 355.515, Validation Accuracy: 0.2316
Fold: 2, Epoch: 11, Train Loss: 332.166, Validation Accuracy: 0.2842
Fold: 2, Epoch: 12, Train Loss: 307.667, Validation Accuracy: 0.3211
Fold: 2, Epoch: 13, Train Loss: 282.125, Validation Accuracy: 0.3868
Fold: 2, Epoch: 14, Train Loss: 254.335, Validation Accuracy: 0.4026
Fold: 2, Epoch: 15, Train Loss: 227.607, Va

Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-67caf77471a5a48e/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-67caf77471a5a48e/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-dcc4d91e0724ffa4.arrow
Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertFor

...

Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-67caf77471a5a48e/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-67caf77471a5a48e/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-dcc4d91e0724ffa4.arrow
Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertFor

Fold: 6, Epoch: 1, Train Loss: 465.518, Validation Accuracy: 0.0000
Fold: 6, Epoch: 2, Train Loss: 462.042, Validation Accuracy: 0.0000
Fold: 6, Epoch: 3, Train Loss: 457.733, Validation Accuracy: 0.0026
Fold: 6, Epoch: 4, Train Loss: 451.263, Validation Accuracy: 0.0053
Fold: 6, Epoch: 5, Train Loss: 440.888, Validation Accuracy: 0.0079
Fold: 6, Epoch: 6, Train Loss: 428.398, Validation Accuracy: 0.0342
Fold: 6, Epoch: 7, Train Loss: 412.714, Validation Accuracy: 0.0789
Fold: 6, Epoch: 8, Train Loss: 394.945, Validation Accuracy: 0.1421
Fold: 6, Epoch: 9, Train Loss: 375.270, Validation Accuracy: 0.2158
Fold: 6, Epoch: 10, Train Loss: 354.141, Validation Accuracy: 0.2868
Fold: 6, Epoch: 11, Train Loss: 330.562, Validation Accuracy: 0.3316
Fold: 6, Epoch: 12, Train Loss: 306.582, Validation Accuracy: 0.3895
Fold: 6, Epoch: 13, Train Loss: 279.342, Validation Accuracy: 0.4132
Fold: 6, Epoch: 14, Train Loss: 252.250, Validation Accuracy: 0.4289
Fold: 6, Epoch: 15, Train Loss: 224.532, Va

Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-67caf77471a5a48e/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-67caf77471a5a48e/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-dcc4d91e0724ffa4.arrow
Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertFor

Fold: 7, Epoch: 1, Train Loss: 465.507, Validation Accuracy: 0.0053
Fold: 7, Epoch: 2, Train Loss: 459.795, Validation Accuracy: 0.0053
Fold: 7, Epoch: 3, Train Loss: 454.869, Validation Accuracy: 0.0026
Fold: 7, Epoch: 4, Train Loss: 447.255, Validation Accuracy: 0.0079
Fold: 7, Epoch: 5, Train Loss: 437.785, Validation Accuracy: 0.0158
Fold: 7, Epoch: 6, Train Loss: 427.548, Validation Accuracy: 0.0369
Fold: 7, Epoch: 7, Train Loss: 412.145, Validation Accuracy: 0.0607
Fold: 7, Epoch: 8, Train Loss: 394.920, Validation Accuracy: 0.1108
Fold: 7, Epoch: 9, Train Loss: 377.001, Validation Accuracy: 0.1715
Fold: 7, Epoch: 10, Train Loss: 356.366, Validation Accuracy: 0.2137
Fold: 7, Epoch: 11, Train Loss: 333.957, Validation Accuracy: 0.2823
Fold: 7, Epoch: 12, Train Loss: 310.023, Validation Accuracy: 0.3034
Fold: 7, Epoch: 13, Train Loss: 286.131, Validation Accuracy: 0.3879
Fold: 7, Epoch: 14, Train Loss: 260.329, Validation Accuracy: 0.4222
Fold: 7, Epoch: 15, Train Loss: 232.782, Va

Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-67caf77471a5a48e/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-67caf77471a5a48e/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-dcc4d91e0724ffa4.arrow
